In [1]:
import pandas as pd
d1 = pd.read_csv('201703-000000000000.csv', compression='gzip')

def clean(line):
    line = line.lower().replace("\n"," ").replace("\r","").replace(',',"").replace(">","> ")
    return line

def file_clean(f):
    data = pd.read_csv(f, compression='gzip')
    cleaned = []
    for line in data['body']:
        cleaned.append(clean(line))
    data['cleaned'] = cleaned
    data['cleaned'].to_csv('pq'+f[-7:])
    return

In [10]:
d2 = d1[:100]
d2.to_csv('sDirtyc.csv.', compression='gzip')

In [29]:
print(d2.columns)

Index(['id', 'title', 'body', 'accepted_answer_id', 'answer_count',
       'comment_count', 'community_owned_date', 'creation_date',
       'favorite_count', 'last_activity_date', 'last_edit_date',
       'last_editor_display_name', 'last_editor_user_id', 'owner_display_name',
       'owner_user_id', 'post_type_id', 'score', 'tags', 'view_count'],
      dtype='object')


In [6]:
# print(d1.iloc[0])
print(clean(d1.iloc[0]['body']))

<p> it seems openstreetmap has changed their licensing scheme as a result lots of data were deleted as shown in the attached picture which is grafton nsw 2460 australia almost all streets are gone.</p>   <p> my question is: is there any way to download the old data somewhere by providing lat/lng's? (i understand that there could be some old archives for world or some countries but that doesn't work for me because at the moment my application is not capable to process those massive data files)</p>   <p> if there's no way to download the old data is there any other good free map data (not images) available?</p>   <p> also i've noticed that there're 4 options at the top right corner the other three except standard seem to be showing all streets. they are (at least mapquest) based on osm data but not the one we get from the "export" section of openstreetmap.org is that correct? </p>   <p> edit: oops as a new user i cannot post images.. the below link may work (or may not):</p>   <p> <a hre

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("Data Clean")\
    .getOrCreate()

In [110]:
dirtyData = spark.read.format("csv.gz").csv("sDirtyc.csv.gz", header=True, multiLine=True, escape='"')
dirtyData.select('id', 'title', 'body','tags').show()

+--------+--------------------+--------------------+--------------------+
|      id|               title|                body|                tags|
+--------+--------------------+--------------------+--------------------+
|11823544|How could I downl...|<p>It seems opens...|       openstreetmap|
|33915105|Get All Records W...|<p>I have a blog ...|           php|mysql|
|14039235|No effect on noti...|<p>I am using a l...|android-notificat...|
|27416240|How to find the p...|<p>I have a prope...|c#|msbuild|msbuil...|
|16236899|Is there an equiv...|<p>I've been usin...|python|python-2.7...|
|26247120|Python - Selectin...|<p>I have a form ...|python|csv|python...|
| 7972886|jaxb annotations ...|<p>For my sins, I...|                jaxb|
|27909083|ConnectivityManag...|<p>I have a quest...|android|android-n...|
|35480928|ALSA unexpected r...|<p>The ALSA lib c...|      linux|gcc|alsa|
| 9436233|Why is System.Typ...|<p>I have a littl...|       c#|reflection|
|22568583|Can ASP.NET Web A...|<p>I ha

In [114]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def clean(line):
    line = line.lower().replace("\n"," ").replace("\r","").replace(',',"").replace(">","> ").replace("<", " <")
    return line
clean_udf = udf(lambda r: clean(r), StringType())

dirtyData = spark.read.csv("sDirtyc.csv.gz", header=True, multiLine=True, escape='"')
# dirtyData.select('id', 'title', 'body','tags').show()
dirtyData = dirtyData.withColumn('cleaned_body', clean_udf(dirtyData['body']))
dirtyData.select('cleaned_body').show()


+--------------------+
|        cleaned_body|
+--------------------+
| <p> it seems ope...|
| <p> i have a blo...|
| <p> i am using a...|
| <p> i have a pro...|
| <p> i've been us...|
| <p> i have a for...|
| <p> for my sins ...|
| <p> i have a que...|
| <p> the alsa lib...|
| <p> i have a lit...|
| <p> i have the f...|
| <p> i am using  ...|
| <p> i am learnin...|
| <p> i have an mv...|
| <p> i'm looking ...|
| <p> i'm looking ...|
| <p> 5 getting er...|
| <p> i've tried s...|
| <p> i started pl...|
| <p> i have a nat...|
+--------------------+
only showing top 20 rows



In [112]:
from pyspark.ml.feature import IDF, IDFModel, Tokenizer
tk = Tokenizer(inputCol='cleaned_body', outputCol='words')
words = tk.transform(dirtyData)
words.select('words').show()
# dirtyData.select('words').show()

+--------------------+
|               words|
+--------------------+
|[, <p>, it, seems...|
|[, <p>, i, have, ...|
|[, <p>, i, am, us...|
|[, <p>, i, have, ...|
|[, <p>, i've, bee...|
|[, <p>, i, have, ...|
|[, <p>, for, my, ...|
|[, <p>, i, have, ...|
|[, <p>, the, alsa...|
|[, <p>, i, have, ...|
|[, <p>, i, have, ...|
|[, <p>, i, am, us...|
|[, <p>, i, am, le...|
|[, <p>, i, have, ...|
|[, <p>, i'm, look...|
|[, <p>, i'm, look...|
|[, <p>, 5, gettin...|
|[, <p>, i've, tri...|
|[, <p>, i, starte...|
|[, <p>, i, have, ...|
+--------------------+
only showing top 20 rows

